In [2]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.callbacks import EarlyStopping
import pandas as pd, numpy as np

# BMI 데이터를 읽어 들이고 정규화하기 --- (※1)
csv = pd.read_csv("bmi.csv")

# 몸무게와 키 데이터
csv["weight"] /= 100   #normalization
csv["height"] /= 200   #normalization
X = csv[["weight", "height"]].as_matrix() # --- (※1a)

# 레이블
bclass = {"thin":[1,0,0], "normal":[0,1,0], "fat":[0,0,1]}
y = np.empty((20000,3))     # 2000x3 크기의 다차원 벡터 생성
for i, v in enumerate(csv["label"]):
    y[i] = bclass[v]        #"thin'이면, y[i]=[1,0,0] 와 같이 할당
    
# 훈련 전용 데이터와 테스트 전용 데이터로 나누기 --- (※2)
X_train, y_train = X[1:15001], y[1:15001]
X_test,  y_test  = X[15001:20001], y[15001:20001] 

# 모델 구조 정의하기 --- (※3)
model = Sequential()  #순차적 계층화 준비
model.add(Dense(512, input_shape=(2,)))  #입력 2개로부터 전달받는 512개 노드의 layer 생성
model.add(Activation('relu'))  #ReLU 활성화함수 채택
model.add(Dropout(0.1))        #dropout ratio=10% (배치 훈련시 10% arc 무시)

model.add(Dense(512))         #512개 노드의 layer 생성
model.add(Activation('relu'))
model.add(Dropout(0.1))

model.add(Dense(3))
model.add(Activation('softmax')) #분류(classification)을 위해 softmax 함수 사용

# 모델 구축하기 --- (※4)
model.compile(
    loss='categorical_crossentropy',  #다중 교차엔트로피
    optimizer="rmsprop",   #최적화 기법 중 하나
    metrics=['accuracy'])  #정확도 측정

# 데이터 훈련하기 --- (※5)
hist = model.fit(
    X_train, y_train,
    batch_size=100,  #100개에 한 번씩 업데이터 실행
    epochs=20,       #훈련 데이터셋을 총 20회 반복 실험. 단, 조기중지될 수 있음
    validation_split=0.1,  
      #validation data 분할 비율. 즉, 15000개 중에서 10%인 1500개를 validation용으로 분할
    callbacks=[EarlyStopping(monitor='val_loss', patience=1)],  
      #'val_loss'를 monitor하여 감소하면 한 번 더(1) 참고 조기중지
    verbose=1)  #전 과정을 화면에 출력(1) 또는 미출력(0) 모드

# 테스트 데이터로 평가하기 --- (※6)
score = model.evaluate(X_test, y_test)
print('loss=', score[0])
print('accuracy=', score[1])

C:\anaconda\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':


Train on 13500 samples, validate on 1500 samples
Epoch 1/20
13500/13500 [==============================] - 1s 65us/step - loss: 0.5174 - acc: 0.7896 - val_loss: 0.3539 - val_acc: 0.8467
Epoch 2/20
13500/13500 [==============================] - 1s 40us/step - loss: 0.2510 - acc: 0.8976 - val_loss: 0.1916 - val_acc: 0.9213
Epoch 3/20
13500/13500 [==============================] - 1s 40us/step - loss: 0.1908 - acc: 0.9182 - val_loss: 0.1291 - val_acc: 0.9560
Epoch 4/20
4999/4999 [==============================] - 0s 34us/step
loss= 0.1737210694421027
accuracy= 0.9077815563351089
